In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from torch.utils.data import TensorDataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader

import torch.optim as optim

import spacy

# NVSM model

In [2]:
class NVSM(nn.Module):
    def __init__(self, n_doc, n_tok, dim_doc_emb, dim_tok_emb, neg_sampling_rate, 
                 pad_token_id):
        super(NVSM, self).__init__()
        self.doc_emb           = nn.Embedding(n_doc, embedding_dim = dim_doc_emb)
        self.tok_emb           = nn.Embedding(n_tok, embedding_dim = dim_tok_emb)
        self.tok_to_doc        = nn.Linear(dim_tok_emb, dim_doc_emb)
        self.bias              = nn.Parameter(torch.Tensor(dim_doc_emb))
        self.neg_sampling_rate = neg_sampling_rate
        self.pad_token_id      = pad_token_id
        
    def query_to_tensor(self, query):
        '''
        Computes the average of the word embeddings of the query. This method 
        corresponds to the function 'g' in the article.
        '''
        # Create a mask to ignore padding embeddings
        query_mask    = (query != self.pad_token_id).float()
        # Compute the number of tokens in each query to properly compute the 
        # average
        tok_by_input  = query_mask.sum(dim = 1)
        query_tok_emb = self.tok_emb(query)
        query_tok_emb = query_tok_emb * query_mask.unsqueeze(-1)
        # Compute the average of the embeddings
        query_emb     = query_tok_emb.sum(dim = 1) / tok_by_input.unsqueeze(-1)
        
        return query_emb
    
    def normalize_query_tensor(self, query_tensor):
        '''
        Divides each query tensor by its L2 norm. This method corresponds to 
        the function 'norm' in the article.
        '''
        norm = torch.norm(query_tensor, dim = 1) # we might have to detach this value 
                                                 # from the computation graph.
        return query_tensor / norm.unsqueeze(-1)
        
    def query_to_doc_space(self, query):
        '''
        Projects a query vector into the document vector space. This method corresponds 
        to the function 'f' in the article.
        '''
        return self.tok_to_doc(query)
    
    def score(self, query, document):
        '''
        Computes the cosine similarity between a query and a document embedding.
        This method corresponds to the function 'score' in the article.
        '''
        # batch dot product using batch matrix multiplication
        num   = torch.bmm(query.unsqueeze(1), document.unsqueeze(-1))
        denum = torch.norm(query, dim = 1) * torch.norm(document, dim = 1)
        
        return num / denum
        
    def non_stand_projection(self, n_gram):
        '''
        Computes the non-standard projection of a n-gram into the document vector 
        space. This method corresponds to the function 'T^~' in the article.
        '''
        n_gram_tensor      = self.query_to_tensor(n_gram)
        norm_n_gram_tensor = self.normalize_query_tensor(n_gram_tensor)
        projection         = self.query_to_doc_space(norm_n_gram_tensor)
        
        return projection
    
    def _custom_batchnorm(self, batch):
        '''
        Computes the variant of the batch normalization formula used in this article. 
        It only uses a bias and no weights.
        '''
        batch_feat_norm = (batch - batch.mean(dim = 0)) / batch.std(dim = 0)
        batch_feat_norm = batch_feat_norm + self.bias
        
        return batch_feat_norm
    
    def stand_projection(self, batch):
        '''
        Computes the standard projection of a n-gram into document vector space with
        a hardtanh activation. This method corresponds to the function 'T' in the 
        article.
        '''
        non_stand_proj = self.non_stand_projection(batch) 
        bn             = self._custom_batchnorm(non_stand_proj)
        activation     = F.hardtanh(bn)

        return activation
    
    def representation_similarity(self, query, document):
        '''
        Computes the similarity between a query and a document. This method corresponds 
        to the function 'P' in the article.
        '''
        document_emb  = self.doc_emb(document)
        query_proj    = self.stand_projection(query)
        # If we have a single document to match against each query, we have
        # to reshape the tensor to compute a simple dot product.
        # Otherwise, we compute a simple matrix multiplication to match the 
        # query against each document.
        if len(document_emb.shape) == 2:
            document_emb = document_emb.unsqueeze(1)
        dot_product   = torch.bmm(document_emb, query_proj.unsqueeze(-1))
        similarity    = torch.sigmoid(dot_product)
        
        return similarity.squeeze()
    
    def forward(self, query, document):
        '''
        Approximates the probability of document given query by uniformly sampling 
        constrastive examples. This method corresponds to the 'P^~' function in the 
        article.
        '''
        # Positive term, this should be maximized as it indicates how similar the
        # correct document is to the query
        pos_repr = self.representation_similarity(query, document)
        
        # Sampling uniformly 'self.neg_sampling_rate' documents to compute the 
        # negative term. We first randomly draw the indices of the documents and 
        # then we compute the similarity with the query.
        z               = self.neg_sampling_rate # corresponds to the z variable in 
                                                 # the article
        n_docs          = self.doc_emb.num_embeddings
        neg_sample_size = (query.size(0), z)
        neg_sample      = torch.randint(low = 0, high = n_docs, size = neg_sample_size)
        neg_repr        = self.representation_similarity(query, neg_sample)
        
        # Probability computation
        positive_term = torch.log(pos_repr)
        negative_term = torch.log(1 - neg_repr).sum(dim = 1)
        proba         = ((z + 1) / (2 * z)) * (z * positive_term + negative_term)
        
        return proba

In [3]:
def loss_function(nvsm, pred, lamb):
    output_term = pred.mean()
    sum_square  = lambda m: (m.weight * m.weight).sum()
    reg_term    = sum_square(nvsm.tok_emb) + \
                  sum_square(nvsm.doc_emb) + \
                  sum_square(nvsm.tok_to_doc)
    loss        = -output_term + (lamb / (2 * pred.shape[0])) * reg_term
    if torch.isnan(loss):
        print('output_term', output_term)
        print('reg_term', reg_term)
    
    return loss

# Dataset creation

In [4]:
spacy_en = spacy.load('en')

def tokenize(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [5]:
def load_docs(filepaths):
    documents = []
    for filepath in filepaths:
        with open(filepath) as file:
            documents.append(file.read().strip().lower())

    return documents

In [6]:
def tokenize_docs(documents):
    tokenized_documents = [tokenize(doc) for doc in documents]
#     print([len(doc_tok) for doc_tok in tokenized_documents])
    
    return tokenized_documents

In [7]:
def create_vocabulary(tokenized_documents):
    vocabulary    = {token for doc in tokenized_documents for token in doc}
    stoi          = {token : i + 2 for i, token in enumerate(vocabulary)}
    stoi['<PAD>'] = 0
    stoi['<UNK>'] = 1
    itos          = {i : token for token, i in stoi.items()}
    
    return vocabulary, stoi, itos

In [8]:
def create_dataset(tok_docs, stoi, n):
    n_grams      = []
    document_ids = []
    for i, doc in enumerate(tok_docs):
        doc_tok_ids = [stoi[tok] for tok in doc]
        for n_gram in [doc_tok_ids[i : i + n] for i in range(len(doc) - n)]:
            n_grams.append(n_gram)
            document_ids.append(i)
            
    return n_grams, document_ids

In [9]:
def create_pytorch_datasets(n_grams, doc_ids, val_prop = 0.2):
    n_grams_tensor = torch.tensor(n_grams)
    doc_ids_tensor = torch.tensor(doc_ids)
    full_dataset   = TensorDataset(n_grams_tensor, doc_ids_tensor)
    total_size     = len(full_dataset)
    val_size       = round(total_size * val_prop)
    train, val     = random_split(full_dataset, [total_size - val_size, val_size])
    
    return train, val

In [10]:
def main():
    filepaths = [
        '../data/raw/language/Word_formation',
        '../data/raw/language/Terminology',    
        '../data/raw/history/Jacobin',
        '../data/raw/history/French_Revolution',
        '../data/raw/math/Game_theory',
        '../data/raw/math/Laplacian_matrix'
    ]
    documents             = load_docs(filepaths)
    tokenized_documents   = tokenize_docs(documents)
    print(len(tokenized_documents))
    voc, stoi, itos       = create_vocabulary(tokenized_documents)
    print(len(stoi))
    n_grams, document_ids = create_dataset(tokenized_documents, stoi, 10)
    train_data, val_data  = create_pytorch_datasets(n_grams, document_ids)

    return stoi, itos, train_data, val_data
stoi, itos, train_data, val_data = main()

6
10811


In [11]:
train_loader = DataLoader(train_data, batch_size = 32, shuffle = True)

In [12]:
nvsm = NVSM(
    n_doc             = 6, 
    n_tok             = 10811, 
    dim_doc_emb       = 20, 
    dim_tok_emb       = 30,
    neg_sampling_rate = 4,
    pad_token_id      = 0
)

In [13]:
optimizer = optim.Adam(nvsm.parameters(), lr = 1e-4)
lamb      = 1e-3 # loss param
#lamb      = 0 # to test if the nan comes from the regularizer

In [14]:
for epoch in range(10):
    for i, (n_grams, doc_ids) in enumerate(train_loader):
        optimizer.zero_grad()
        pred_proba = nvsm(n_grams, doc_ids)
        loss = loss_function(nvsm, pred_proba, lamb)
        if torch.isnan(loss):
            break
        loss.backward()
        optimizer.step()
        if i % 250 == 0:
            print(f'[{epoch},{i}]: {loss}')

[0,0]: 11.626873016357422
[0,250]: 9.820734024047852
[0,500]: 10.764383316040039
[0,750]: 9.480093002319336
[0,1000]: 9.724170684814453
[0,1250]: 9.760310173034668
[0,1500]: 8.451714515686035
[0,1750]: 9.364665985107422
[1,0]: 8.12890338897705
[1,250]: 8.304265975952148
[1,500]: 8.27496337890625
[1,750]: 8.218366622924805
[1,1000]: 7.938173294067383
[1,1250]: 7.983478546142578
[1,1500]: 8.036639213562012
[1,1750]: 7.432046890258789
[2,0]: 7.266980171203613
[2,250]: 7.686492919921875
[2,500]: 7.563785076141357
[2,750]: 7.225085258483887
[2,1000]: 7.2503180503845215
[2,1250]: 7.513890266418457
[2,1500]: 7.1929931640625
output_term tensor(-7.6769, grad_fn=<MeanBackward1>)
reg_term tensor(nan, grad_fn=<AddBackward0>)
output_term tensor(-7.7120, grad_fn=<MeanBackward1>)
reg_term tensor(nan, grad_fn=<AddBackward0>)
output_term tensor(-5.8969, grad_fn=<MeanBackward1>)
reg_term tensor(nan, grad_fn=<AddBackward0>)
output_term tensor(-5.5434, grad_fn=<MeanBackward1>)
reg_term tensor(nan, grad_fn

In [15]:
torch.any(torch.isnan(nvsm.tok_emb.weight))

tensor(1, dtype=torch.uint8)

In [19]:
weight = nvsm.tok_emb.weight
c = 0
for i in range(weight.shape[0]):
    if torch.any(torch.isnan(weight[i])):
        print(itos[i])
#        print(weight[i])
        c += 1

principles



italians
90px
are
exile
payoff
0
on
their
overview==
's
anchor|economics
'
i
.
constant
soboul
521
new
"
k.m
consists
|
approach
learning
:
66627
theory
fête
foreign
finally
antoinette
not
de
needed|date
as
summer
games
select
was
reforms
=
sans
revolution
>
bergen.jpg|thumb|upright=1.15|[[french
{
economics
london
country
equilibrium
sum
systems
by
speaking&nbsp;–
chooses
whose
j
itself.<ref>{{cite
agrarian
dummy
,
|year=2008
business===
cancelled


player
that
had
shus-6>{{nl
overthrow
primarily
politics
franse
if
monarchy
l'etre
terminology
2001
be
 
players
newspapers.{{citation
suprême
each
the
bartlett
game
magnitude
\end{array}\right)</math
matrix
out
york
associates
falling
peasants
baker
]
cult
routledge
correspond
}
ministers
board
2
36#2
experimental
shusterman
journal
\neq
5|page=233}}</ref
thus
behavior
many
1\\
about
s
king
for
comparatively
early
history
opposition
choices
characterised
ed
murders
leaders
called
and
noah
names
particular
file
work
itself
&


In [18]:
c

195